In [2]:
import os
import glob
import pathlib

In [3]:
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC

In [5]:
this_dir = pathlib.Path(os.path.abspath(""))

In [6]:
data_dir = this_dir / "data"

Configure the browser

In [7]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")

In [8]:
driver = webdriver.Chrome(options=options)

Get the page

In [9]:
driver.get(
    "https://datavisualization.cdph.ca.gov/t/LNC/views/COVIDSNFDASHV3/COVIDSNFDASH?:embed=y&:showVizHome=no&:host_url=https%3A%2F%2Fdatavisualization.cdph.ca.gov%2F&:embed_code_version=3&:tabs=yes&:toolbar=yes&:showAppBanner=false&:display_spinner=no&:loadOrderID=0"
)

Pull the date

In [10]:
def get_date():
    wait = WebDriverWait(driver, 300)
    wait.until(
        EC.visibility_of_element_located(
            (By.ID, "tableau_base_widget_QuickFilterPanel_0")
        )
    )
    date_filter = driver.find_element_by_id("tableau_base_widget_QuickFilterPanel_0")
    date_string = date_filter.find_elements_by_xpath(
        "//div[contains(@class, 'tabComboBoxName')]"
    )[-1].text
    return pd.to_datetime(date_string).date()

In [11]:
date = get_date()

Routine to pull the download button

In [12]:
def get_download_button():
    wait = WebDriverWait(driver, 300)
    wait.until(EC.visibility_of_element_located((By.ID, "download-ToolbarButton")))
    btn = driver.find_element_by_id("download-ToolbarButton")
    assert btn.text == "Download"
    return btn

Routine to pull data for values

In [13]:
def get_data(element_id):
    # WAit for the box
    wait = WebDriverWait(driver, 300)
    wait.until(EC.visibility_of_element_located((By.ID, element_id)))

    # Get it and click it twice
    box = driver.find_element_by_id(element_id)
    box.click()
    box.click()

    # Click the download button
    download_button = get_download_button()
    download_button.click()

    # Wait for the popup
    wait.until(EC.visibility_of_element_located((By.XPATH, "//fieldset")))
    fieldset = driver.find_element_by_xpath("//fieldset")

    # Get the Data button and click it
    buttons = fieldset.find_elements_by_xpath("//button")
    data_button = next(e for e in buttons if e.text == "Data")
    data_button.click()

    # Move to the new tab
    driver.switch_to.window(driver.window_handles[1])

    # Get the data
    wait.until(EC.visibility_of_element_located((By.XPATH, "//td")))
    value = int(driver.find_element_by_xpath("//td").text)

    # Close out
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    # Return the data
    return value

Pull the four we're after

In [14]:
staff_active_cases = get_data("tabZoneId74")

In [15]:
patients_active_cases = get_data("tabZoneId124")

In [16]:
patients_deaths = get_data("tabZoneId78")

In [17]:
staff_deaths = get_data("tabZoneId86")

In [18]:
patients_confirmed_cases = get_data("tabZoneId76")

In [19]:
staff_confirmed_cases = get_data("tabZoneId84")

In [20]:
data = dict(
    date=date,
    staff_active_cases=staff_active_cases,
    patients_active_cases=patients_active_cases,
    patients_confirmed_cases=patients_confirmed_cases,
    patients_deaths=patients_deaths,
    staff_confirmed_cases=staff_confirmed_cases,
    staff_deaths=staff_deaths,
)

In [21]:
data

{'date': datetime.date(2021, 9, 15),
 'staff_active_cases': 332,
 'patients_active_cases': 534,
 'patients_confirmed_cases': 63455,
 'patients_deaths': 9200,
 'staff_confirmed_cases': 53889,
 'staff_deaths': 249}

Kill the browser

In [22]:
driver.quit()

Export

In [23]:
df = pd.DataFrame([data])

In [24]:
df.to_csv(data_dir / f"{date}.csv", index=False)

In [25]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [26]:
df_list = []
for csv in csv_list:
    file_date = csv.split("/")[-1].replace(".csv", "")
    df = pd.read_csv(csv, parse_dates=["date"])
    df_list.append(df)

In [27]:
df = pd.concat(df_list).sort_values(["date"])

In [28]:
df.to_csv(data_dir / "timeseries.csv", index=False)